In [ ]:
import pathlib
import os
import warnings

from notebooks.hybrid_ts import sensitivity

warnings.filterwarnings('ignore')

# Change to the project root directory
project_root = pathlib.Path().resolve().parent
os.chdir(project_root)

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from src.train import CrossValidator
from src.train import Trainer
from src.models import Classic_TCN
from src.diagnose import compute_erf

# Import Data

In [ ]:
data_path = pathlib.Path("data/DCOILWTICO.csv")
print(f"Loading from: {data_path.absolute()}")
print(f"File exists: {data_path.exists()}")

data = pd.read_csv(data_path)
data.rename(columns = {"observation_date" : "date", "DCOILWTICO" : "price"}, inplace  = True)
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")
data = data.asfreq('B')
data["price"] = data["price"].fillna(method='ffill')
data["return"] = np.log(data["price"]) - np.log(data["price"].shift(1))
returns = data["return"].replace([np.inf, -np.inf], np.nan).dropna().astype("float32")

# Split Data

In [ ]:
# Three-way split: 60% train, 20% validation, 20% test
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

train_end = int(len(returns) * train_ratio)
val_end = int(len(returns) * (train_ratio + val_ratio))

# Split the data
y_train = returns.iloc[:train_end]
y_val = returns.iloc[train_end:val_end]
y_test = returns.iloc[val_end:]

# Convert to darts TimeSeries objects
train_series = TimeSeries.from_values(y_train)
val_series = TimeSeries.from_values(y_val)
test_series = TimeSeries.from_values(y_test)

print(f"Train size: {len(train_series)} ({train_ratio*100}%)")
print(f"Validation size: {len(val_series)} ({val_ratio*100}%)")
print(f"Test size: {len(test_series)} ({(1-train_ratio-val_ratio)*100}%)")

# Plot Data

In [ ]:
#Plot of returns and data split
fig, ax = plt.subplots(figsize=(14, 6))
#Split the data
train_data = returns.iloc[:train_end]
val_data = returns.iloc[train_end:val_end]
test_data = returns.iloc[val_end:]

#plot each section with different colors
ax.plot(train_data.index, train_data.values,
        linewidth=0.8, color='#2E86AB', alpha=0.8, label='Training Set')
ax.plot(val_data.index, val_data.values,
        linewidth=0.8, color='#A23B72', alpha=0.8, label='Validation Set')
ax.plot(test_data.index, test_data.values,
        linewidth=0.8, color='#F18F01', alpha=0.8, label='Test Set')

#add horizontal line at zero
ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.4)

#add vertical lines to separate sections
ax.axvline(x=train_data.index[-1], color='gray', linestyle='--', linewidth=1.5, alpha=0.5)
ax.axvline(x=val_data.index[-1], color='gray', linestyle='--', linewidth=1.5, alpha=0.5)

ax.set_title('WTI Crude Oil Daily Log Returns with Data Splits',
             fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Log Return', fontsize=12)
ax.legend(loc='upper right', framealpha=0.95, fontsize=10)
ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Cross-Validate TCN

In [ ]:
#define hyperparameter grid
hyperparameter_grid = {
    'kernel_size': [2, 3, 5],
    'num_filters': [64, 128],
    'num_layers': [3, 5, 7],
    'dilation_base': [2, 4],
    'lr': [0.0001, 0.001, 0.01]
}

#run cross-validation
cv = CrossValidator(
    model_type="classic",
    hyperparameter_grid=hyperparameter_grid,
    num_epochs=100,
    seed=42
)
cv.fit(train_series, val_series)
best_config_tcn = cv.get_best_config()
results_tcn = cv.get_results()
model_tcn = cv.get

# Train TCN

In [ ]:
try:
    # Only pass architecture parameters here
    model_classic_tcn = Classic_TCN(
        num_channels=[128, 128, 128],
        kernel_size=[2, 2, 2],
        dilations=[1, 2, 4],
        dropout=0.1
    )

    model_classic_tcn.load_state_dict(torch.load("weights/model_weights_classic_tcn.pth"))
    model_classic_tcn.eval()
    print(f"Loaded model weights from 'weights/model_weights_classic_tcn.pth'")

except (RuntimeError, FileNotFoundError) as e:
    print(f"Model architecture mismatch: {e}")
    print(f"Training new model...")
    # Training parameters go here
    model_classic_tcn = Trainer(
        model_type="classic",
        num_channels=[128, 128, 128],
        kernel_size=[2, 2, 2],
        dilations=[1, 2, 4],
        num_epochs=100,
        lr=0.0001
    ).fit(y_train.values)
    torch.save(model_classic_tcn.state_dict(), "weights/model_weights_classic_tcn.pth")